# Testing notebook

In [14]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor
from writing import *

## We need some testing data

In [15]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

## Computions funcitons

In [16]:
from computions import *

### get_describe_nominal function test

Basic situation

In [17]:
get_describe_nominal(test_frame.iloc[:,1], Y)

,count,1.0,1.0%,2.0,2.0%,0.0,0.0%
object_variable,,,,,,,
9.0,113,61,53.982301,52.0,46.017699,0.0,0.000000
1.0,105,11,10.476190,0.0,0.000000,94.0,89.523810
5.0,105,87,82.857143,0.0,0.000000,18.0,17.142857
7.0,103,95,92.233010,7.0,6.796117,1.0,0.970874
6.0,98,87,88.775510,4.0,4.081633,7.0,7.142857
3.0,98,44,44.897959,0.0,0.000000,54.0,55.102041
2.0,95,23,24.210526,0.0,0.000000,72.0,75.789474
4.0,92,68,73.913043,0.0,0.000000,24.0,26.086957
8.0,89,65,73.033708,24.0,26.966292,0.0,0.000000


Data frame with emptys

In [18]:
test = get_describe_nominal(na_containts_frame.iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))
test = get_describe_nominal(na_containts_frame.fillna('empty').iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))

Is all obs. are in table -  False
Is all obs. are in table -  True


### AUC computing test

AUC numeric - simple fucntion

In [19]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{1.0: 0.5225439012814428, 2.0: 0.7661557244918398, 0.0: 0.3467313435570457}
{'category 1': 0.5225439012814428, 'category 2': 0.7661557244918398, 'cetegory 0': 0.3467313435570457}
{'category 1': 0.5253192389006343, 'category 2': 0.7666755801170633, 'cetegory 0': 0.3423416087483415}
{1.0: 0.6532686564429542, 0.0: 0.3467313435570457}


AUC nominal

In [20]:
non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys - needs to be replaces in previous steps')
print(get_AUC_nominal(na_containts_frame['object_variable'].fillna('empty'), Y_strs))

get_AUC_nominal(na_containts_frame['object_variable'].fillna('empty'), Y_strs, 
                get_describe_nominal(na_containts_frame.iloc[:,1].fillna('empty'), Y_strs))


{'category 1': 0.8389272226481529, 'category 2': 0.9248848233958736, 'cetegory 0': 0.9364201432785504}
AUC bynary
{1.0: 0.9364201432785503, 0.0: 0.9364201432785504}
AUC with emptys - needs to be replaces in previous steps
{'category 1': 0.8365846040264644, 'category 2': 0.9246749778231383, 'cetegory 0': 0.9334313563824914}


{'category 1': 0.8365846040264644,
 'category 2': 0.9246749778231383,
 'cetegory 0': 0.9334313563824914}

Get publish AUC for any predictor type

In [21]:
non_table = get_full_AUC(test_frame['object_variable'], Y, 'nominal')
print('nominal predictor without table')
print(non_table)

with_table = get_full_AUC(test_frame['object_variable'], Y_strs, 'nominal', descr_table = dn_tab)
print('moninal predictor with table')
print(with_table)

numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
print('numeric predictor')
print(numeric_var)

nominal predictor without table
{1.0: {'AUC': 0.8389272226481529, 'rel_type': 1, 'GINI': 0.6778544452963058}, 2.0: {'AUC': 0.9248848233958736, 'rel_type': 1, 'GINI': 0.8497696467917473}, 0.0: {'AUC': 0.9364201432785504, 'rel_type': 1, 'GINI': 0.8728402865571008}}
moninal predictor with table
{'category 1': {'AUC': 0.8389272226481529, 'rel_type': 1, 'GINI': 0.6778544452963058}, 'category 2': {'AUC': 0.9248848233958736, 'rel_type': 1, 'GINI': 0.8497696467917473}, 'cetegory 0': {'AUC': 0.9364201432785504, 'rel_type': 1, 'GINI': 0.8728402865571008}}
numeric predictor
{1.0: {'AUC': 0.5225439012814428, 'rel_type': 1, 'GINI': 0.04508780256288558}, 2.0: {'AUC': 0.7661557244918398, 'rel_type': 1, 'GINI': 0.5323114489836795}, 0.0: {'AUC': 0.6532686564429543, 'rel_type': -1, 'GINI': 0.3065373128859086}}


Decoding AUC info as DataFrame

In [22]:
numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
AUC_info_to_DataFrame(numeric_var,'numeric_variable')

1.0                          2.0                     \
                       AUC rel_type      GINI       AUC rel_type      GINI   
numeric_variable  0.522544        1  0.045088  0.766156        1  0.532311   

                       0.0                     
                       AUC rel_type      GINI  
numeric_variable  0.653269       -1  0.306537

get_all_computions funcion test

In [23]:
print('describe_table with no nas')
print(get_all_comuptions(test_frame['object_variable'], Y)['describe_table'])
print('describe_table with nas')
print(get_all_comuptions(na_containts_frame['object_variable'], Y)['describe_table'])

describe_table with no nas
                 count  1.0       1.0%   2.0       2.0%   0.0       0.0%
object_variable                                                         
9.0                113   61  53.982301  52.0  46.017699   0.0   0.000000
1.0                105   11  10.476190   0.0   0.000000  94.0  89.523810
5.0                105   87  82.857143   0.0   0.000000  18.0  17.142857
7.0                103   95  92.233010   7.0   6.796117   1.0   0.970874
6.0                 98   87  88.775510   4.0   4.081633   7.0   7.142857
3.0                 98   44  44.897959   0.0   0.000000  54.0  55.102041
2.0                 95   23  24.210526   0.0   0.000000  72.0  75.789474
4.0                 92   68  73.913043   0.0   0.000000  24.0  26.086957
8.0                 89   65  73.033708  24.0  26.966292   0.0   0.000000
0.0                 53    1   1.886792   0.0   0.000000  52.0  98.113208
10.0                49   17  34.693878  32.0  65.306122   0.0   0.000000
describe_table with nas


Getting indicators from computions uotput

In [24]:
comp_result =  get_all_comuptions(test_frame['numeric_variable'], Y)
print('full numeric data')
get_predictor_row(comp_result)
print('numeric data with nas, with str categoryes')
comp_result =  get_all_comuptions(na_containts_frame['numeric_variable'], Y_strs)
get_predictor_row(comp_result)

full numeric data
numeric data with nas, with str categoryes


category 1                    category 2                     \
                        AUC rel_type      GINI        AUC rel_type      GINI   
numeric_variable   0.525319        1  0.050638   0.766676        1  0.533351   

                 cetegory 0                    Emptys Emptys part  
                        AUC rel_type      GINI                     
numeric_variable   0.657658       -1  0.315317     20        0.02

## Writing to excel

Adding a info about different predictors

In [25]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()

Let's show final table

In [26]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
#my_cpp.get_predictors_data()
my_cpp.result_DF

1.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.522544        1  0.045088   
object_variable                                0.838927        1  0.677854   
column with soo000oo000oo000oo000oo long name  0.507409       -1  0.014818   

                                                    2.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.766156        1  0.532311   
object_variable                                0.924885        1   0.84977   
column with soo000oo000oo000oo000oo long name  0.512042        1  0.024085   

                                                    0.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.653269       -1  0.306537   
object_variable                                 0.93642        1   0.87284   
column with soo000oo000oo000oo000oo long name  0.502583        1  0.005167   

                                              Emptys Emptys part  
                                                                  
numeric_variable                                   0         0.0  
object_variable                                    0         0.0  
column with soo000oo000oo000oo000oo long name      0         0.0

Writig a double header table

In [27]:


xl_writer = pd.ExcelWriter("test_result/double_header_saver.xlsx",engine='xlsxwriter')
ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws      

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

print_double_column_header(  my_cpp, ws, 
                            my_cpp.result_DF.loc[['numeric_variable'],:],
                            "A1", 'Hello World', 
                            xl_writer.book.add_format(my_cpp.default_header_format))

xl_writer.close()

Writing info about some predictor

In [28]:
xl_writer = pd.ExcelWriter("test_result/writing_a_value.xlsx",engine='xlsxwriter')

ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws                             

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

default_predictor_printer(my_cpp, ws, my_cpp.get_predictors_data()['numeric_variable'], 
{'header_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'desc_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'class_ab_format':xl_writer.book.add_format(my_cpp.default_header_format)})

xl_writer.close()

In [29]:
xl_writer = pd.ExcelWriter("test_result/result_test.xlsx",engine='xlsxwriter')
my_cpp.write_to_book(xl_writer)
xl_writer.close()